In [1]:
import psycopg2
import requests
import pandas as pd
from sqlalchemy import create_engine

In [4]:
dbname = "kucoin"
user = "postgres"
password = "6962277"
host = "localhost"  # Puedes cambiarlo si la base de datos no está en la misma máquina
port = "5432"  # El puerto predeterminado de PostgreSQL


In [5]:
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = conn.cursor()
    print("Conexión exitosa")
    # Puedes ejecutar consultas SQL aquí utilizando el cursor
except Exception as e:
    print(f"Error de conexión: {e}")


Conexión exitosa


In [6]:

# URL de la API para obtener la lista de monedas
api_url = 'https://api.kucoin.com/api/v3/currencies'

# Realizar una solicitud GET a la API de KuCoin
response = requests.get(api_url)

# Verificar si la solicitud se completó con éxito (código de respuesta 200)
if response.status_code == 200:
    data = response.json()  # Obtener los datos de la respuesta en formato JSON
    currencies = data.get('data', [])  # Obtener la lista de monedas desde los datos

    currency_data_list = []  # Crear una lista para almacenar los datos de las monedas

    for currency in currencies:
        currency_data = {
            'currency': currency['currency'],
            'name': currency['name'],
            'fullName': currency['fullName'],
            'precision': currency['precision'],
            'isMarginEnabled': currency['isMarginEnabled'],
            'isDebitEnabled': currency['isDebitEnabled']
        }
        currency_data_list.append(currency_data)  # Agregar los datos de la moneda a la lista

        print(f"Currency: {currency['currency']}")
        print(f"Name: {currency['name']}")
        print(f"Full Name: {currency['fullName']}")
        print(f"Precision: {currency['precision']}")
        print(f"Is Margin Enabled: {currency['isMarginEnabled']}")
        print(f"Is Debit Enabled: {currency['isDebitEnabled']}")
        print("----------")

    # Ahora, puedes insertar currency_data_list en tu base de datos PostgreSQL
    # Usando psycopg2 o la biblioteca de tu elección

else:
    print("Error al obtener la lista de monedas")


Currency: CSP
Name: CSP
Full Name: Caspian
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: False
----------
Currency: LOKI
Name: OXEN
Full Name: Oxen
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: True
----------
Currency: NRG
Name: NRG
Full Name: Energi
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: True
----------
Currency: HOT
Name: HOT
Full Name: HotNow
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: False
----------
Currency: FET
Name: FET
Full Name: Fetch.Ai
Precision: 8
Is Margin Enabled: True
Is Debit Enabled: True
----------
Currency: XMR
Name: XMR
Full Name: Monero
Precision: 8
Is Margin Enabled: True
Is Debit Enabled: True
----------
Currency: RBTC
Name: RBTC
Full Name: RSK Smart Bitcoin
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: True
----------
Currency: RIF
Name: RIF
Full Name: RIF Token
Precision: 8
Is Margin Enabled: False
Is Debit Enabled: True
----------
Currency: ANKR
Name: ANKR
Full Name: Ankr Network
Precision: 8
Is Mar

In [7]:

# Aquí creamos un DataFrame a partir de los datos
df = pd.DataFrame(currency_data_list)

# Realizar operaciones de transformación en el DataFrame
df['name'] = df['name'].str.lower()
df['fullName'] = df['fullName'].str.lower()


# Almacenar temporalmente en csv
df.to_csv('monedas.csv', index=False)



# Ahora, el DataFrame "df" contiene los datos de las monedas después de las transformaciones necesarias



In [8]:
try:
    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS coin_list (
            currency VARCHAR PRIMARY KEY,
            name VARCHAR,
            fullName VARCHAR,
            precision INT,
            isMarginEnabled BOOLEAN,
            isDebitEnabled BOOLEAN
        )
    """)
    conn.commit()  # Guardar los cambios en la base de datos
    print("Tabla 'coin_list' creada o ya existe")

except Exception as e:
    print(f"Error al crear la tabla: {e}")



Tabla 'coin_list' creada o ya existe


In [9]:
try:
    # Insertar nuevas monedas sin duplicados
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO coin_list (currency, name, fullName, precision, isMarginEnabled, isDebitEnabled)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (currency) DO NOTHING
        """, (row['currency'], row['name'], row['fullName'], row['precision'], row['isMarginEnabled'], row['isDebitEnabled']))
    
    # Confirmar la transacción
    conn.commit()
    print("Datos insertados exitosamente")

except Exception as e:
    conn.rollback()  # Deshacer cambios en caso de error
    print(f"Error de conexión: {e}")



Datos insertados exitosamente


In [17]:
# Obtener las monedas desde la base de datos
try:
    cursor.execute("SELECT currency FROM coin_list")
    coins = cursor.fetchall()
except Exception as e:
    print(f"Error al obtener monedas desde la base de datos: {e}")

# Definir el intervalo de tiempo
interval = "1week"

# Lista para almacenar los datos históricos
historical_data = []

# Lista para almacenar las monedas que tienen datos históricos
remaining_coins = []

# Iterar a través de las monedas y obtener datos históricos
for coin in coins:
    currency = coin[0]
    symbol = f"{currency}-USDT"

    # URL de la API de KuCoin para obtener datos históricos de precios
    url = f"https://api.kucoin.com/api/v1/market/candles?type={interval}&symbol={symbol}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data["code"] == "200000":
                historical_prices = data["data"]
                if historical_prices:
                    for price_data in historical_prices:
                        timestamp = price_data[0]  # Fecha y hora en formato UNIX timestamp
                        open_price = price_data[1]  # Precio de apertura
                        high_price = price_data[2]  # Precio máximo
                        low_price = price_data[3]  # Precio mínimo
                        close_price = price_data[4]  # Precio de cierre
                        volume = price_data[5]  # Volumen de operaciones

                        # Agregar una columna para la moneda correspondiente
                        price_data.append(currency)

                        historical_data.append(price_data)
                    remaining_coins.append(currency)
                else:
                    print(f"No se encontraron datos históricos para {currency}.")
            else:
                print(f"No se encontraron datos históricos para {currency}")
        else:
            print(f"Error al obtener datos históricos para {currency}: Código de estado {response.status_code}")
    except Exception as e:
        print(f"Error de conexión para {currency}: {e}")

# Eliminar monedas que no tienen datos históricos de la base de datos
if remaining_coins:
    cursor.execute("DELETE FROM coin_list WHERE currency NOT IN %s", (tuple(remaining_coins),))
    conn.commit()



Error al obtener datos históricos para LAVAX: Código de estado 429
Error al obtener datos históricos para BCH: Código de estado 429


In [18]:
# Crear un DataFrame a partir de los datos históricos
columns = ["timestamp", "open", "high", "low", "close", "volume", "assetvolume", "currency"]
historical_df = pd.DataFrame(historical_data, columns=columns)

# Guardar el DataFrame en un archivo CSV
historical_df.to_csv("historical_data_week.csv", index=False)


In [19]:
# Lee el archivo CSV
df = pd.read_csv("historical_data_week.csv")

# Convierte el timestamp de UNIX timestamp a formato de fecha y hora
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Guarda los datos en un nuevo archivo CSV
df.to_csv("historical_data_week2.csv", index=False)

In [20]:

# Consulta SQL para crear una nueva tabla (ajústala según tus necesidades)
create_table_query = """
CREATE TABLE IF NOT EXISTS historical_data_week (
    timestamp TIMESTAMP,
    open NUMERIC,
    high NUMERIC,
    low NUMERIC,
    close NUMERIC,
    volume NUMERIC,
    assetvolume NUMERIC,
    currency VARCHAR
);
"""

# Ejecuta la consulta para crear la nueva tabla
cursor.execute(create_table_query)
conn.commit()

# Utiliza pandas para leer los archivos CSV en DataFrames
historical_data_df = pd.read_csv("historical_data_week2.csv")

# Crea una conexión de motor para SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Inserta los datos de los DataFrames en la nueva tabla
historical_data_df.to_sql("historical_data_week", engine, if_exists="append", index=False)



618

In [ ]:
cursor.close()
conn.close()
